In [78]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, accuracy_score, roc_auc_score, log_loss

from sklearn.model_selection import train_test_split

from src.data_prep import prepare_data
# from src.models import logistic_regression_model, decision_tree_model, random_forest_model, lightgbm_model
# from src.train import train_sklearn_model
from src.evaluation import evaluate_model

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import joblib

import torch
import torch.nn as nn
import torch.optim as optim

from imblearn.over_sampling import SMOTE

import lightgbm as lgb

sns.set(style="whitegrid")

In [112]:
df = pd.read_csv('data/creditcard/creditcard.csv')
X = df.drop(columns='Class')
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    random_state=42,
    test_size=0.3
)

X_fit, X_val, y_fit, y_val = train_test_split(
    X_train, y_train,
    random_state=42
)

X_train.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
2557,2105.0,-2.289565,-0.480260,0.818685,-1.706423,0.822102,-1.660326,0.944047,-0.541765,1.323156,...,-0.831985,-0.210837,0.914737,0.867888,0.422969,0.310584,-0.781488,0.392241,-0.147757,1.00
247823,153702.0,-0.313717,-4.064342,-3.398445,0.704011,0.101662,1.529848,1.551670,-0.036774,0.015829,...,2.142593,0.853186,-0.091941,-0.936215,-0.833081,-0.498728,0.651183,-0.290331,0.110360,1194.28
152342,97283.0,-1.809763,-0.567439,2.265186,-0.960318,-1.212537,1.516493,-1.417176,0.903421,1.961027,...,-0.554004,-0.509915,-0.424978,-0.268621,0.010121,0.466862,0.835540,-0.062385,0.088079,75.00
103385,68628.0,1.192319,0.178575,0.141491,0.459628,-0.049959,-0.112122,-0.163883,0.155740,-0.067566,...,-0.149985,-0.240464,-0.739862,0.116799,-0.373837,0.125470,0.130126,-0.016956,0.011937,1.98
8771,11951.0,-0.963451,0.700311,1.097333,-1.547626,0.669966,0.513533,0.333683,0.270900,1.381880,...,0.122458,-0.279519,-0.470181,-0.124037,-1.388839,-0.237453,0.785347,0.349708,0.216207,37.31


In [113]:
fit = lgb.Dataset(X_fit, y_fit)
val = lgb.Dataset(X_val, y_val, reference=fit)

model = lgb.train(
    params={
        'learning_rate': 0.01,
        'objective': 'binary'
    },
    train_set=fit,
    num_boost_round=400,
    valid_sets=(fit, val),
    valid_names=('fit', 'val'),
  callbacks=[
    lgb.early_stopping(stopping_rounds=20),
    lgb.log_evaluation(period=100)
    ]
)

y_pred = model.predict(X_test)

print()
print(f"Test's ROC AUC: {roc_auc_score(y_test, y_pred):.5f}")
print(f"Test's logloss: {log_loss(y_test, y_pred):.5f}")

[LightGBM] [Info] Number of positive: 267, number of negative: 149256
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010222 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 149523, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001786 -> initscore=-6.326170
[LightGBM] [Info] Start training from score -6.326170
Training until validation scores don't improve for 20 rounds
[100]	fit's binary_logloss: 0.00195632	val's binary_logloss: 0.00401289
[200]	fit's binary_logloss: 0.000826295	val's binary_logloss: 0.00350349
Early stopping, best iteration is:
[272]	fit's binary_logloss: 0.000490346	val's binary_logloss: 0.00342967

Test's ROC AUC: 0.97806
Test's logloss: 0.00207


In [114]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

optimal_idx = (tpr - fpr).argmax()
optimal_threshold = thresholds[optimal_idx]

y_pred_binary = (y_pred >= optimal_threshold).astype(int)
# y_pred_binary = (y_pred >= 0.5).astype(int)

# Save binary predictions and true labels
# binary_predictions = pd.DataFrame({"y_true": y_test, "y_pred": y_pred_binary})
# binary_predictions.to_csv("artifacts/predictions/lightgbm_bce_predictions.csv", index=False)

predictions = pd.DataFrame({"y_true": y_test, "y_pred": y_pred})
predictions.to_csv("artifacts/lightgbm_preds/lightgbm_bce_continuous_predictions.csv", index=False)

# print(f"Optimal Threshold: {optimal_threshold:.5f}")
print("Binary predictions saved successfully!")

Binary predictions saved successfully!


In [115]:
import numpy as np
from scipy import optimize
from scipy import special

class FocalLoss:

    def __init__(self, gamma, alpha=None):
        self.alpha = alpha
        self.gamma = gamma

    def at(self, y):
        if self.alpha is None:
            return np.ones_like(y)
        return np.where(y, self.alpha, 1 - self.alpha)

    def pt(self, y, p):
        p = np.clip(p, 1e-15, 1 - 1e-15)
        return np.where(y, p, 1 - p)

    def __call__(self, y_true, y_pred):
        at = self.at(y_true)
        pt = self.pt(y_true, y_pred)
        return -at * (1 - pt) ** self.gamma * np.log(pt)

    def grad(self, y_true, y_pred):
        y = 2 * y_true - 1  # {0, 1} -> {-1, 1}
        at = self.at(y_true)
        pt = self.pt(y_true, y_pred)
        g = self.gamma
        return at * y * (1 - pt) ** g * (g * pt * np.log(pt) + pt - 1)

    def hess(self, y_true, y_pred):
        y = 2 * y_true - 1  # {0, 1} -> {-1, 1}
        at = self.at(y_true)
        pt = self.pt(y_true, y_pred)
        g = self.gamma

        u = at * y * (1 - pt) ** g
        du = -at * y * g * (1 - pt) ** (g - 1)
        v = g * pt * np.log(pt) + pt - 1
        dv = g * np.log(pt) + g + 1

        return (du * v + u * dv) * y * (pt * (1 - pt))

    def init_score(self, y_true):
        res = optimize.minimize_scalar(
            lambda p: self(y_true, p).sum(),
            bounds=(0, 1),
            method='bounded'
        )
        p = res.x
        log_odds = np.log(p / (1 - p))
        return log_odds

    def lgb_obj(self, preds, train_data):
        y = train_data.get_label()
        p = special.expit(preds)
        return self.grad(y, p), self.hess(y, p)

    def lgb_eval(self, preds, train_data):
        y = train_data.get_label()
        p = special.expit(preds)
        is_higher_better = False
        return 'focal_loss', self(y, p).mean(), is_higher_better

In [116]:
fl = FocalLoss(alpha=None, gamma=0)

fit = lgb.Dataset(
    X_fit, y_fit,
    init_score=np.full_like(y_fit, fl.init_score(y_fit), dtype=float)
)

val = lgb.Dataset(
    X_val, y_val,
    init_score=np.full_like(y_val, fl.init_score(y_fit), dtype=float),
    reference=fit
)

model = lgb.train(
    params={
        'learning_rate': 0.01,
        'objective': fl.lgb_obj
    },
    train_set=fit,
    num_boost_round=10000,
    valid_sets=(fit, val),
    valid_names=('fit', 'val'),
    callbacks=[
        lgb.early_stopping(stopping_rounds=20),
        lgb.log_evaluation(period=100)
    ],
    feval=fl.lgb_eval
)

y_pred = special.expit(fl.init_score(y_fit) + model.predict(X_test))

print()
print(f"Test's ROC AUC: {roc_auc_score(y_test, y_pred):.5f}")
print(f"Test's logloss: {log_loss(y_test, y_pred):.5f}")

[LightGBM] [Info] Using self-defined objective function
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011780 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 149523, number of used features: 30
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 20 rounds
[100]	fit's focal_loss: 0.00195348	val's focal_loss: 0.00401427
[200]	fit's focal_loss: 0.00082718	val's focal_loss: 0.00351291
Early stopping, best iteration is:
[272]	fit's focal_loss: 0.000489416	val's focal_loss: 0.00344263

Test's ROC AUC: 0.97987
Test's logloss: 0.00208


In [117]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

optimal_idx = (tpr - fpr).argmax()
optimal_threshold = thresholds[optimal_idx]

y_pred_binary = (y_pred >= optimal_threshold).astype(int)

# y_pred_binary = (y_pred >= 0.5).astype(int)

# binary_predictions = pd.DataFrame({"y_true": y_test, "y_pred": y_pred_binary})
# binary_predictions.to_csv("artifacts/predictions/lightgbm_focal_loss_predictions.csv", index=False)

predictions = pd.DataFrame({"y_true": y_test, "y_pred": y_pred})
predictions.to_csv("artifacts/lightgbm_preds/lightgbm_focal_loss_continuous_predictions.csv", index=False)

print(f"Optimal Threshold: {optimal_threshold:.5f}")
print("Binary predictions saved successfully!")

Optimal Threshold: 0.00506
Binary predictions saved successfully!


## ROC-Star

In [118]:
class ROCStarLoss:

    def __init__(self, delta=2, gamma=0.4):

        self.delta = delta

        self.gamma = gamma

        self.epoch_true = None

        self.epoch_pred = None

        self.bce_epoch = 0

        self.BCE = nn.BCELoss()

 

    def calc_loss(self, y_true, y_pred):

       

        # If first epoch, BCE Loss

        if self.bce_epoch > 0:

            self.bce_epoch -= 1

            self.epoch_true = y_true.clone()

            self.epoch_pred = y_pred.clone()

 

            return self.BCE(y_pred, y_true)

 

        # B/W cmparison for Appeal/No Appeal - [B(+), W(-)]

 

        pos_ind = y_true >= 0.5

        neg_ind = y_true < 0.5

 

        B = y_pred[pos_ind]

        W = y_pred[neg_ind]

 

        B_shifted = B - self.gamma

 

        # Batch-wise loss calculatoin

        batch_size = 50000

        loss = 0.0

 

        for i in range(0, len(W), batch_size):

            W_batch = W[i:i + batch_size]

 

            comparisons_batch = W_batch.unsqueeze(1) - B_shifted.unsqueeze(0)

 

            loss += torch.sum((torch.clamp(comparisons_batch, min=0))**2)

 

        # end code #

 

        # full dataset loss

        # comparisons = W.unsqueeze(1) - B_shifted.unsqueeze(0)

 

        # loss_matrix = torch.clamp(comparisons, min=0)

        # loss2 = torch.sum(loss_matrix ** 2)

        # print("Losses for full and batch-wise")

        # print(loss)

        # print(loss2)

        # end code #

 

        return loss

 

    def init_score(self, y_true):

        p = np.mean(y_true)

        p = np.clip(p, 1e-15, 1 - 1e-15)

        log_odds = np.log(p / (1 - p))

        return log_odds

 

    def grad(self, y_true, y_pred):

        B_ind = y_true >= 0.5

        W_ind = y_true < 0.5

 

        B = y_pred[B_ind]

        W = y_pred[W_ind]

 

        dB = B * (1 - B)

        dW = W * (1 - W)

 

        # Batching for GPU Memory saving

        batch_size = 50000

 

        sum_comparisons_B = torch.zeros_like(B)

        sum_comparisons_W = torch.zeros_like(W)

 

        for i in range(0, len(W), batch_size):

            W_batch = W[i: i + batch_size]

 

            comparisons_B_batch = W_batch.unsqueeze(1) - B.unsqueeze(0) + self.gamma

 

            sum_comparisons_B_batch = torch.clamp(comparisons_B_batch, min=0).sum(dim=0)

            sum_comparisons_W_batch = torch.clamp(comparisons_B_batch, min=0).sum(dim=1)

 

            sum_comparisons_B += sum_comparisons_B_batch

            sum_comparisons_W[i:i + batch_size] += sum_comparisons_W_batch

 

        dLdx = -2 * sum_comparisons_B * dB

        dLdy = 2 * sum_comparisons_W * dW

 

        # end code #

 

        # Normal matrix code for full dataset

        # comparisons = W.unsqueeze(1) - B.unsqueeze(0) + self.gamma

        # comparisons = torch.clamp(comparisons, min=0)

 

        # dLdx2 = -2 * torch.sum(comparisons, dim=0) * dB

        # dLdy2 = 2 * torch.sum(comparisons, dim=1) * dW

 

        # print('Sums')

        # print(torch.all(torch.isclose(dLdx, dLdx2)))

        # print(torch.all(torch.isclose(dLdy, dLdy2)))

        # end code #

 

        dL = torch.zeros_like(y_pred)

        dL[B_ind] = dLdx

        dL[W_ind] = dLdy

 

        return dL

 

 

    def hess(self, y_true, y_pred):

        B_ind = y_true >= 0.5

        W_ind = y_true < 0.5

 

        B = y_pred[B_ind]

        W = y_pred[W_ind]

 

        B_hessian = torch.zeros_like(B)

        W_hessian = torch.zeros_like(W)

 

        batch_size = 20000

 

        for i in range(0, len(W), batch_size):

            W_batch = W[i:i + batch_size]

 

            margin_matrix_batch = W_batch.unsqueeze(1) - B.unsqueeze(0) + self.gamma

            hessian_matrix_batch = torch.clamp(margin_matrix_batch, min=0)

 

            sum_dx_batch = torch.sum(hessian_matrix_batch, dim=0)

            sum_dy_batch = torch.sum(hessian_matrix_batch, dim=1)

 

            B_hessian += -2*B*(1-B)*sum_dx_batch

            B_hessian += 4*B*((1-B)**2)*sum_dx_batch

            B_hessian += torch.sum(2*((B*(1-B))**2) * (hessian_matrix_batch != 0), dim=0)

 

            W_hessian[i:i + batch_size] += 2*W_batch*(1-W_batch)*sum_dy_batch

            W_hessian[i:i + batch_size] += -4*W_batch*((1-W_batch)**2)*sum_dy_batch

            W_hessian[i:i + batch_size] += torch.sum(2*((W_batch*(1-W_batch))**2) * (hessian_matrix_batch != 0).T, dim=0)

 

        # Entire Dataset

 

        # margin_matrix = W.unsqueeze(1) - B.unsqueeze(0) + self.gamma

        # hessian_matrix = torch.clamp(margin_matrix, min=0)

 

        # sum_dx = torch.sum(hessian_matrix, dim=0)

        # sum_dy = torch.sum(hessian_matrix, dim=1)

 

        # B_hessian2 = -2*B*(1-B)*sum_dx + 4*B*((1-B)**2)*sum_dx + torch.sum(2*((B*(1-B))**2) * (hessian_matrix != 0), dim=0)

        # W_hessian2 = 2*W*(1-W)*sum_dy - 4*W*((1-W)**2)*sum_dy + torch.sum(2*((W*(1-W))**2) * (hessian_matrix != 0).T, dim=0)

 

        # end code #

 

        # print('Hessians')

        # if not torch.all(torch.isclose(B_hessian, B_hessian2, atol=1e-1)).item():

        #     print(B_hessian)

        #     print(B_hessian2)

        # print(torch.all(torch.isclose(W_hessian, W_hessian2)))

 

        hessians = torch.zeros_like(y_pred)

        hessians[B_ind] = B_hessian

        hessians[W_ind] = W_hessian

 

        return hessians

 

 

    def calc_grad_hess(self, y_true, y_pred):

        y_true = torch.tensor(y_true, dtype=torch.float32).cuda()

        y_pred = torch.tensor(y_pred, dtype=torch.float32).cuda()

 

        grad = self.grad(y_true, y_pred)

        hess = self.hess(y_true, y_pred)

 

        # del y_true

        # del y_pred

 

        grad = grad.cpu().detach().numpy()

        hess = hess.cpu().detach().numpy()

 

        # return grad, np.ones(grad.shape)

        return grad, hess

 

    def rocstar_obj(self, preds, train_data):

        y = train_data.get_label()

        p = special.expit(preds)

 

        grad, hess = self.calc_grad_hess(y, p)

 

        return grad, hess

 

    def rocstar_eval(self, preds, train_data):

        y = train_data.get_label()

        p = special.expit(preds)

 

        loss_metric = 'bce_loss' if self.bce_epoch > 0 else 'rocstar_loss'

        loss = self.calc_loss(torch.tensor(y, dtype=torch.float32).cuda(), torch.tensor(p, dtype=torch.float32).cuda())

        is_higher_better = False

 

        return loss_metric, loss.item(), is_higher_better

In [127]:
def train_model(X_train, y_train, X_fit, X_val, y_fit, y_val):
    fit = lgb.Dataset(X_fit, y_fit, free_raw_data=False)
    val = lgb.Dataset(X_val, y_val, reference=fit, free_raw_data=False)
    gamma = 1
    
    rocstar = ROCStarLoss(gamma=gamma)

    params = {
        'learning_rate': 0.03,
        'num_leaves': 100,
        'gamma': gamma,
        'is_unbalance': True,
        'objective': rocstar.rocstar_obj
    }

    print('#######   Training LightGBM with roc-star   #######')
    model = lgb.train(
        params=params,
        train_set=fit,
        num_boost_round=10000,
        valid_sets=(fit, val),
        valid_names=('fit', 'val'),
        callbacks=[
            lgb.early_stopping(stopping_rounds=300),
            lgb.log_evaluation(period=100)
        ],
        feval=rocstar.rocstar_eval,
    )
    return model

In [128]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    random_state=42,
    test_size=0.3
)

X_fit, X_val, y_fit, y_val = train_test_split(
    X_train, y_train,
    random_state=42
)

rocstar_model = train_model(X_train, y_train, X_fit, X_val, y_fit, y_val)

#######   Training LightGBM with roc-star   #######
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Info] Using self-defined objective function
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012587 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 149523, number of used features: 30
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Info] Using self-defined objective function
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 300 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warni

In [129]:
def predict_results(model, X_test, y_test):
    y_pred = special.expit(model.predict(X_test))
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)

    optimal_idx = (tpr - fpr).argmax()
    optimal_threshold = thresholds[optimal_idx]
    
    y_pred_binary = (y_pred >= optimal_threshold).astype(int)
    # y_pred_binary = (y_pred > 0.5).astype(int)
    print(f"Test's ROC AUC: {roc_auc_score(y_test, y_pred):.5f}")
    print(f"Test's logloss: {log_loss(y_test, y_pred):.5f}")

    return y_pred, y_pred_binary

In [130]:
y_pred, y_pred_binary = predict_results(model, X_test, y_test)

# binary_predictions = pd.DataFrame({"y_true": y_test, "y_pred": y_pred_binary})
# binary_predictions.to_csv("artifacts/predictions/lightgbm_rocstar_predictions.csv", index=False)

predictions = pd.DataFrame({"y_true": y_test, "y_pred": y_pred})
predictions.to_csv("artifacts/lightgbm_preds/lightgbm_rocstar_continuous_predictions.csv", index=False)

Test's ROC AUC: 0.97987
Test's logloss: 0.09871
